## TD ESN BY UM :Collecte des Données en Python pour le Traitement du Langage Naturel (NLP)

### API key is: 551f227a8b304114bd1103fe63642e9a

In [10]:
import requests
import pandas as pd

# Base URL de l'API PubMed
base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"

# Paramètres pour rechercher des articles sur les AVC
params = {
    "db": "pubmed",          # Base de données PubMed
    "term": "stroke",        # Mot-clé de recherche
    "retmax": 10,            # Limiter à 10 résultats
    "retmode": "json",       # Format de la réponse
}

# Effectuer la requête à l'API PubMed, on a pas besoin de notre clé API pour PUBMED. 
response = requests.get(base_url, params=params)

if response.status_code == 200:
    # Un code 200 signifie que la requête a réussi. 
    data = response.json() # Une réponse JSON (JavaScript Object Notation)
    articles = data.get("esearchresult", {}).get("idlist", [])
    
    if articles:#verification de la presence des articles
        print(f"{len(articles)} articles trouvés : {articles}")

        # URL de l'API pour récupérer les détails des articles
        details_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
        details_params = {
            "db": "pubmed",           # Base de données
            "id": ",".join(articles), # Liste des IDs des articles
            "retmode": "json",        # Format de la réponse
        }

        # Requête pour obtenir les détails des articles
        details_response = requests.get(details_url, params=details_params)

        if details_response.status_code == 200:
            details_data = details_response.json()
            articles_summary = details_data.get("result", {})
            
            # Extraire les articles, exclure l'élément "uids" (liste d'ID), uids est un élément qui contient une liste d'ID uniques des articles retournés par la requête.
            article_list = [
                {
                    "Titre": article.get("title", "Titre non disponible"),
                    "Journal": article.get("source", "Journal non disponible"),
                    "Date de publication": article.get("pubdate", "Date non disponible"),
                    "URL PubMed": f"https://pubmed.ncbi.nlm.nih.gov/{article_id}"
                }
                for article_id, article in articles_summary.items()
                if article_id != "uids"
            ]

            # Afficher les résultats
            for idx, article in enumerate(article_list, start=1):
                print(f"\nArticle {idx} :")
                print(f"Titre : {article['Titre']}")
                print(f"Journal : {article['Journal']}")
                print(f"Date de publication : {article['Date de publication']}")
                print(f"URL : {article['URL PubMed']}")

            # Sauvegarder les résultats dans un fichier CSV
            output_file = "articles_pubmed_stroke.csv"
            df = pd.DataFrame(article_list)
            df.to_csv(output_file, index=False, encoding="utf-8")
            print(f"\nLes articles ont été sauvegardés dans le fichier {output_file}.")
        else:
            print(f"Erreur lors de la récupération des détails : {details_response.status_code}")
    else:
        print("Aucun article trouvé pour ce mot-clé.")
else:
    print(f"Erreur lors de la requête API : {response.status_code} - {response.text}")


10 articles trouvés : ['39690389', '39690049', '39690002', '39689934', '39689929', '39689886', '39689714', '39689705', '39689634', '39689578']

Article 1 :
Titre : Profiling immune cell-related gene features and immunoregulatory ceRNA in ischemic stroke.
Journal : Mol Biomed
Date de publication : 2024 Dec 18
URL : https://pubmed.ncbi.nlm.nih.gov/39690389

Article 2 :
Titre : Association between handgrip strength and mortality risk among cardiovascular disease patients: A cohort study from the NHANES 2011-2014.
Journal : J Formos Med Assoc
Date de publication : 2024 Dec 16
URL : https://pubmed.ncbi.nlm.nih.gov/39690049

Article 3 :
Titre : Effect of continuous positive airway pressure on non-fatal stroke and paroxysmal atrial fibrillation recurrence in obstructive sleep apnoea elderly patients.
Journal : Eur J Intern Med
Date de publication : 2024 Dec 16
URL : https://pubmed.ncbi.nlm.nih.gov/39690002

Article 4 :
Titre : The efficacy of colchicine compared to placebo for preventing isch